In [297]:
import pandas as pd
import numpy as np
import csv 
import matplotlib.pyplot as plt
import spacy
import math
import os
import statsmodels.api as sm

from src.utils import *
from src.data_preprocessing import *
from src.data_processing import *
from src.feature_selection import *
from src.modelisation import *

os.chdir('/Users/ismailakrout/Desktop/python/NLP_statapp')

pd.set_option("display.max_columns", None)


In [298]:
# pour ne pas charger le NB avec des warnings 
import warnings 
warnings.filterwarnings('ignore')

In [299]:
df = pd.read_csv('01. output/aux.csv')

In [300]:
df.drop(columns=['Unnamed: 0', 'text'], inplace=True)

In [301]:
aux = pd.melt(df, id_vars=['Speaker', 'party'])

In [302]:
aux.sort_values(by=['variable'], inplace=True)
aux = aux.reset_index(drop=True)

In [303]:
lox_de_lox = aux[['Speaker', 'party', 'value']].groupby(by=['Speaker', 'party']).sum().reset_index()
lox_de_lox.rename(columns={'value':'m_it'},inplace=True)

In [304]:
aux = pd.merge(
    aux,
    lox_de_lox,
    how='left',
    on=['Speaker', 'party']
)

In [305]:
# On Calcule les termes ds le sigma 
aux['q_hat_it'] = aux['value']/aux['m_it']

In [306]:
aux_Lab = aux.loc[aux['party'] == 'Lab']
aux_Con = aux.loc[aux['party'] == 'Con']

In [307]:
lox_de_lox = aux_Con[['variable', 'value']].groupby(by=['variable']).sum().reset_index()
lox_de_lox.rename(columns={'value':'Somme_c_it'},inplace=True)
aux_Con = pd.merge(
    aux_Con,
    lox_de_lox,
    how='left',
    on=['variable']
)
aux_Con['Somme_c_it'] = aux_Con['Somme_c_it'] - aux_Con['value']
lox_de_lox = aux.drop_duplicates(subset=['Speaker'])
lox_de_lox['Somme_m_it'] = lox_de_lox['m_it'].sum()
aux_Con = pd.merge(
    aux_Con,
    lox_de_lox[['Speaker', 'Somme_m_it']],
    how='left',
    on=['Speaker']
)
aux_Con['Somme_m_it'] = aux_Con['Somme_m_it'] - aux_Con['m_it']
aux_Con['q_hat_Con_-ijt'] = aux_Con['Somme_c_it'] / aux_Con['Somme_m_it']

In [308]:
lox_de_lox = aux_Lab[['variable', 'value']].groupby(by=['variable']).sum().reset_index()
lox_de_lox.rename(columns={'value':'Somme_c_it'},inplace=True)
aux_Lab = pd.merge(
    aux_Lab,
    lox_de_lox,
    how='left',
    on=['variable']
)
aux_Lab['Somme_c_it'] = aux_Lab['Somme_c_it'] - aux_Lab['value']
lox_de_lox = aux.drop_duplicates(subset=['Speaker'])
lox_de_lox['Somme_m_it'] = lox_de_lox['m_it'].sum()
aux_Lab = pd.merge(
    aux_Lab,
    lox_de_lox[['Speaker', 'Somme_m_it']],
    how='left',
    on=['Speaker']
)
aux_Lab['Somme_m_it'] = aux_Lab['Somme_m_it'] - aux_Lab['m_it']
aux_Lab['q_hat_Lab_-ijt'] = aux_Lab['Somme_c_it'] / aux_Lab['Somme_m_it']

In [309]:
aux_Lab_intial = aux_Lab.drop(columns=['q_hat_Lab_-ijt'])
aux_Lab_intial['Somme_m_it'] = aux_Lab_intial['Somme_m_it'] + aux_Lab_intial['m_it']
aux_Lab_intial['Somme_c_it'] = aux_Lab_intial['Somme_c_it'] + aux_Lab_intial['value']
aux_Lab_intial['q_hat_Lab_jt'] = aux_Lab_intial['Somme_c_it'] / aux_Lab_intial['Somme_m_it']
aux_Lab_intial = aux_Lab_intial[['variable', 'q_hat_Lab_jt']]
aux_Lab_intial.drop_duplicates(subset=['variable'], inplace=True)

In [310]:
# La je fais mon merge 
aux_Con = pd.merge(
    aux_Con,
    aux_Lab_intial,
    how='left',
    on=['variable']
)
aux_Con['rho_hat_-ijt'] = (aux_Con['q_hat_Con_-ijt']) / (aux_Con['q_hat_Con_-ijt'] + aux_Con['q_hat_Lab_jt'])
aux_Con['produit'] = aux_Con['q_hat_it'] * aux_Con['rho_hat_-ijt']
nb_Con = len(aux_Con['Speaker'].unique())
res_Con = aux_Con[['variable', 'produit']].groupby(by=['variable']).sum().reset_index()
res_Con['moitie_Con_pi'] = res_Con['produit']*(1/2)*(1/nb_Con)
res_Con.drop(columns=['produit'], inplace=True)

## je refais pour les Labors 

In [311]:
aux_Con_intial = aux_Con.drop(columns=['q_hat_Con_-ijt'])
aux_Con_intial['Somme_m_it'] = aux_Con_intial['Somme_m_it'] + aux_Con_intial['m_it']
aux_Con_intial['Somme_c_it'] = aux_Con_intial['Somme_c_it'] + aux_Con_intial['value']
aux_Con_intial['q_hat_Con_jt'] = aux_Con_intial['Somme_c_it'] / aux_Con_intial['Somme_m_it']
aux_Con_intial = aux_Con_intial[['variable', 'q_hat_Con_jt']]
aux_Con_intial.drop_duplicates(subset=['variable'], inplace=True)

In [312]:
#  La je fais mon merge 
aux_Lab = pd.merge(
    aux_Lab,
    aux_Lab_intial,
    how='left',
    on=['variable']
)
aux_Lab['rho_hat_-ijt'] = (aux_Lab['q_hat_Lab_-ijt']) / (aux_Lab['q_hat_Lab_-ijt'] + aux_Lab['q_hat_Lab_jt'])
aux_Lab['produit'] = aux_Lab['q_hat_it'] * aux_Lab['rho_hat_-ijt']
nb_Lab = len(aux_Lab['Speaker'].unique())
res_Lab = aux_Lab[['variable', 'produit']].groupby(by=['variable']).sum().reset_index()
res_Lab['moitie_Lab_pi'] = res_Lab['produit']*(1/2)*(1/nb_Lab)
res_Lab.drop(columns=['produit'], inplace=True)

In [315]:
res = pd.merge(
    res_Con,
    res_Lab,
    how='left',
    on=['variable'],
)

In [317]:
res['pi'] = res['moitie_Con_pi'] + res['moitie_Lab_pi']

In [318]:
res

,variable,moitie_Con_pi,moitie_Lab_pi,pi
0,abil,0.000364,0.000237,0.000601
1,abl,0.002495,0.001939,0.004434
2,absolut,0.000720,0.000358,0.001078
3,accept,0.000479,0.000522,0.001001
4,achiev,0.000389,0.000286,0.000676
...,...,...,...,...
495,withdraw,0.000137,0.000106,0.000243
496,word,0.000279,0.000383,0.000662
497,write,0.000090,0.000091,0.000180
498,yes,0.000303,0.000296,0.000599
